# Assignment: Feature Engineering 2 Module 16 Checkpoint 8 - Jaz Viccarro

In [5]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
from sqlalchemy import create_engine
import warnings


import math
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


%matplotlib inline

warnings.filterwarnings('ignore')
sns.set(style="whitegrid")

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

df = pd.read_sql_query('select * from useducation',con=engine)

engine.dispose()

In [7]:
df.head().T

,0,1,2,3,4
PRIMARY_KEY,1992_ALABAMA,1992_ALASKA,1992_ARIZONA,1992_ARKANSAS,1992_CALIFORNIA
STATE,ALABAMA,ALASKA,ARIZONA,ARKANSAS,CALIFORNIA
YEAR,1992,1992,1992,1992,1992
ENROLL,NaN,NaN,NaN,NaN,NaN
TOTAL_REVENUE,2.67888e+06,1.04959e+06,3.25808e+06,1.71196e+06,2.626e+07
FEDERAL_REVENUE,304177,106780,297888,178571,2.07247e+06
STATE_REVENUE,1.65903e+06,720711,1.36982e+06,958785,1.65465e+07
LOCAL_REVENUE,715680,222100,1.59038e+06,574603,7.64104e+06
TOTAL_EXPENDITURE,2.6538e+06,972488,3.40158e+06,1.74302e+06,2.71388e+07
INSTRUCTION_EXPENDITURE,1.4817e+06,498362,1.43591e+06,964323,1.43589e+07


In [9]:
missing = pd.DataFrame(df.isnull().sum() * 100 / len(df), columns=["Missing Values %"])
missing = missing.sort_values(by="Missing Values %", ascending=False)
missing = missing.round(0)
missing = missing[missing['Missing Values %'] != 0]

missing_vals_cols = ['ENROLL',
 'OTHER_EXPENDITURE',
 'INSTRUCTION_EXPENDITURE',
 'SUPPORT_SERVICES_EXPENDITURE',
 'CAPITAL_OUTLAY_EXPENDITURE',
 'TOTAL_EXPENDITURE',
 'LOCAL_REVENUE',
 'STATE_REVENUE',
 'FEDERAL_REVENUE',
 'TOTAL_REVENUE',
 'GRADES_PK_G',
 'GRADES_ALL_G',
 'GRADES_KG_G',
 'GRADES_4_G',
 'GRADES_8_G',
 'GRADES_12_G',
 'GRADES_1_8_G',
 'GRADES_9_12_G']

df1 = df.copy()

states = df1['STATE'].unique() # use state as the index

for state in states:
    df1.loc[df1["STATE"] == state, missing_vals_cols] = df1.loc[df1["STATE"] == state, missing_vals_cols].interpolate()

df1.dropna(inplace=True)

#### 1. Create a new score variable from the weighted averages of all score variables in the datasets. Notice that the number of students in the 4th grade isn't the same as the number of students in the 8th grade. So, you should appropriately weigh the scores!.

In [26]:
df1.shape #25 cols
df1.head()
scores_df = df1.iloc[:,15:25] #isolating for better view, personal preference
scores_df.head(13)

for column in scores_df:
    print(len(scores_df[column])) # just checking

466
466
466
466
466
466
466
466
466
466


In [37]:
df1['weighted_score'] = (df1["GRADES_4_G"]*((df1["AVG_MATH_4_SCORE"] + df1["AVG_READING_4_SCORE"])*0.5) + df1["GRADES_8_G"]
                                 * ((df1["AVG_MATH_8_SCORE"] + df1["AVG_READING_8_SCORE"])*0.5))/(df1["GRADES_4_G"] + df1["GRADES_8_G"])

New column created above
________________________________

#### 2. What are the correlations between this newly created score variable and the expenditure types? Which 1 of the expenditure types is more correlated than the others?

In [43]:
df1[["weighted_score", "TOTAL_EXPENDITURE", "INSTRUCTION_EXPENDITURE",
              "SUPPORT_SERVICES_EXPENDITURE", "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]].corr().round(2) 

,weighted_score,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
weighted_score,1.00,0.07,0.09,0.08,0.01,-0.01
TOTAL_EXPENDITURE,0.07,1.00,0.99,0.99,0.94,0.93
INSTRUCTION_EXPENDITURE,0.09,0.99,1.00,0.97,0.90,0.88
SUPPORT_SERVICES_EXPENDITURE,0.08,0.99,0.97,1.00,0.95,0.91
OTHER_EXPENDITURE,0.01,0.94,0.90,0.95,1.00,0.92
CAPITAL_OUTLAY_EXPENDITURE,-0.01,0.93,0.88,0.91,0.92,1.00


Weighted Score is most positively correlated with Instruction Expenditure, then Support Services Expenditures, then Total Expenditure.

#### 3. Now, apply PCA to the 4 expenditure types. How much of the total variance is explained by the 1st component?


In [45]:
X = df1[["TOTAL_EXPENDITURE", "INSTRUCTION_EXPENDITURE", 
         "SUPPORT_SERVICES_EXPENDITURE", "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]]

X = StandardScaler().fit_transform(X) # data already normalized
Xt = X.T
Cx = np.cov(Xt)
print('Covariance Matrix:\n', Cx)


Covariance Matrix:
 [[1.00215054 0.9923841  0.99512353 0.94573151 0.92777464]
 [0.9923841  1.00215054 0.97649399 0.90411344 0.88670093]
 [0.99512353 0.97649399 1.00215054 0.95433021 0.91320483]
 [0.94573151 0.90411344 0.95433021 1.00215054 0.92016275]
 [0.92777464 0.88670093 0.91320483 0.92016275 1.00215054]]


In [52]:
sklearn_pca = PCA(n_components=1)
df1['PCA_1'] = sklearn_pca.fit_transform(X)


print(
    'The percentage of total variance in the dataset explained by each',
    'component from Sklearn PCA.\n',
    (sklearn_pca.explained_variance_ratio_)*100
)

The percentage of total variance in the dataset explained by each component from Sklearn PCA.
 [95.19222218]


> Answer: 95% of the total variance in the dataset is explained by the First Principal Component.

____________________________

#### 4. What is the correlation between the overall score variable and the 1st principal component? 


In [53]:
df1[["weighted_score", "PCA_1", "TOTAL_EXPENDITURE", "INSTRUCTION_EXPENDITURE",
              "SUPPORT_SERVICES_EXPENDITURE", "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]].corr().round(2) 

,weighted_score,PCA_1,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
weighted_score,1.00,0.05,0.07,0.09,0.08,0.01,-0.01
PCA_1,0.05,1.00,1.00,0.97,0.99,0.97,0.95
TOTAL_EXPENDITURE,0.07,1.00,1.00,0.99,0.99,0.94,0.93
INSTRUCTION_EXPENDITURE,0.09,0.97,0.99,1.00,0.97,0.90,0.88
SUPPORT_SERVICES_EXPENDITURE,0.08,0.99,0.99,0.97,1.00,0.95,0.91
OTHER_EXPENDITURE,0.01,0.97,0.94,0.90,0.95,1.00,0.92
CAPITAL_OUTLAY_EXPENDITURE,-0.01,0.95,0.93,0.88,0.91,0.92,1.00


#### >Answer: The Overall Score and 1st principal component are positively but weakly correlated.

#### 5. If you were to choose the best variables for your model, would you prefer using the 1st principal component instead of the expenditure variables? Why?

#### >Answer: I'd prefer to use the most highly correlated varibale, Instruction Expenditure, over the Principal Component variable.